# Import dependencies

In [24]:
import csv
import cv2
import fnmatch
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Dense, Flatten
from keras.models import model_from_json, Sequential
from keras.optimizers import Adam
import math
from matplotlib import pyplot
import numpy as np
import os
import scipy
from sklearn.utils import shuffle

# Set parameters

In [25]:
CAMERA_COUNT = 3

DRIVING_LOG_PATH = './data'
DRIVING_LOG_FILE = 'driving_log.csv'

CENTER_IMAGE_REGULAR_EXPRESSION = 'center*'
IMAGE_PATH = './data/IMG'

SET_SIZE = 512
EPOCH = 4
VALIDATION_SET_SIZE = 0.2

center_camera_file = 'center_2016_12_01_13_37_16_570.jpg'
left_camera_file = 'left_2016_12_01_13_37_16_570.jpg'
right_camera_file = 'right_2016_12_01_13_37_16_570.jpg'

hard_left_file = 'center_2016_12_01_13_39_28_024.jpg'
hard_right_file = 'center_2016_12_01_13_38_46_752.jpg'

# Get data

Use data by Udacity.

Source: https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip

Size: 322.8 MB

In [26]:
# Use beta simulator to gather data using mouse

In [27]:
# Get at least 40,000 samples

In [28]:
# Use generator to read data

In [29]:
# Resize image

In [30]:
# Edit steering angle for left and right camera images

In [31]:
# Get features and labels that correspond to hard left turn, straight, and hard right turn
# Tune model to predict these 3 steering angles correctly
# left = np.array(cv2.imread(
#     os.path.join(
#         IMAGE_PATH,
#         hard_left_file
#     )
# ), dtype = 'float32')
# center = np.array(cv2.imread(
#     os.path.join(
#         IMAGE_PATH,
#         center_camera_file
#     )
# ), dtype = 'float32')
# right = np.array(cv2.imread(
#     os.path.join(
#         IMAGE_PATH,
#         hard_right_file
#     )
# ), dtype = 'float32')
# features = np.stack((left, center, right))

# labels = np.array([-0.9426954, 0, 1], dtype = 'float32')

In [ ]:
file = open(os.path.join(DRIVING_LOG_PATH, DRIVING_LOG_FILE), 'r')
reader = csv.reader(file)

reader.__next__()

features = np.array(
    [
        cv2.imread(
            os.path.join(
                IMAGE_PATH,
                reader
                    .__next__()[0]
                    .strip('IMG/')
            )
        )
        for index in range(SET_SIZE)
    ],
    dtype = 'float32'
)

In [ ]:
file = open(os.path.join(DRIVING_LOG_PATH, DRIVING_LOG_FILE), 'r')
reader = csv.reader(file)

reader.__next__()

labels = np.array(
    [
        reader.__next__()[3]
        for index in range(SET_SIZE)
    ],
    dtype = 'float32'
)

In [40]:
def generate_sample():
    file = open(os.path.join(DRIVING_LOG_PATH, DRIVING_LOG_FILE), 'r')
    reader = csv.reader(file)
    
    reader.__next__()
    
    while True:
        line = reader.__next__()
        
        yield np.array(
            cv2.imread(
                os.path.join(
                    IMAGE_PATH,
                    line[0].strip('IMG/')
                )
            )
        )
       

In [41]:
generator = generate_sample()

image = next(generator)
print(image.shape)

(160, 320, 3)


# Explore data

In [32]:
file = open(os.path.join(DRIVING_LOG_PATH, DRIVING_LOG_FILE), 'r')
reader = csv.reader(file)

print(reader.__next__())
print(reader.__next__())

['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']
['IMG/center_2016_12_01_13_30_48_287.jpg', ' IMG/left_2016_12_01_13_30_48_287.jpg', ' IMG/right_2016_12_01_13_30_48_287.jpg', ' 0', ' 0', ' 0', ' 22.14829']


In [ ]:
count = len(os.listdir(IMAGE_PATH))

print('Count:', count)
print('Frames:', count / CAMERA_COUNT)

In [ ]:
data = np.array([1 for file in os.listdir(IMAGE_PATH) if fnmatch.fnmatch(file, CENTER_IMAGE_REGULAR_EXPRESSION)], dtype = 'float32')
print('Center image count:', data.sum())

In [ ]:
image_center = cv2.imread(os.path.join(IMAGE_PATH, center_camera_file))
print('Shape:', image_center.shape)

In [ ]:
pyplot.imshow(cv2.cvtColor(image_center, cv2.COLOR_BGR2RGB))
pyplot.show()

In [ ]:
image_left = cv2.imread(os.path.join(IMAGE_PATH, left_camera_file))
pyplot.imshow(cv2.cvtColor(image_left, cv2.COLOR_BGR2RGB))
pyplot.show()

In [ ]:
image_right = cv2.imread(os.path.join(IMAGE_PATH, right_camera_file))
pyplot.imshow(cv2.cvtColor(image_right, cv2.COLOR_BGR2RGB))
pyplot.show()

In [ ]:
image_hard_left = cv2.imread(os.path.join(IMAGE_PATH, hard_left_file))
pyplot.imshow(cv2.cvtColor(image_hard_left, cv2.COLOR_BGR2RGB))
pyplot.show()

In [ ]:
image_hard_right = cv2.imread(os.path.join(IMAGE_PATH, hard_right_file))
pyplot.imshow(cv2.cvtColor(image_hard_right, cv2.COLOR_BGR2RGB))
pyplot.show()

In [ ]:
scipy.stats.describe(labels)

In [ ]:
pyplot.hist(labels)
pyplot.show()


# Transform data

In [ ]:
image_center_resized = cv2.resize(image_center, (32, 16))
print('Shape:', image_center_resized.shape)

In [ ]:
pyplot.imshow(cv2.cvtColor(image_center_resized, cv2.COLOR_BGR2RGB))
pyplot.show()

In [ ]:
y_start = 64
y_end = image_center.shape[0] - 30
x_start = 60
x_end = image_center.shape[1] - 60

image_center_cropped = image_center[y_start:y_end, x_start:x_end]
print('Shape:', image_center_cropped.shape)

In [ ]:
pyplot.imshow(cv2.cvtColor(image_center_cropped, cv2.COLOR_BGR2RGB))
pyplot.show()

In [ ]:
image_center_cropped_grayscale = cv2.cvtColor(image_center_cropped, cv2.COLOR_BGR2GRAY)

In [ ]:
pyplot.imshow(image_center_cropped_grayscale, cmap = 'gray')
pyplot.show()

In [ ]:
image_center_cropped_grayscale_resized = cv2.resize(image_center_cropped_grayscale, (32, 8))

In [ ]:
pyplot.imshow(image_center_cropped_grayscale_resized, cmap = 'gray')
pyplot.show()

# Process data

In [ ]:
def normalize(image_data):
    lower_bound = -0.5
    upper_bound = 0.5
    minimum_pixel = 0
    maximum_pixel = 255
    
    return lower_bound + (
        (image_data - minimum_pixel) *
        (upper_bound - lower_bound) /
        (maximum_pixel - minimum_pixel)
    )

normalized_features = normalize(features)

In [ ]:
epsilon = 1e-5

assert math.isclose(
    np.min(normalized_features),
    -0.5,
    abs_tol = epsilon
) and math.isclose(
    np.max(normalized_features),
    0.5,
    abs_tol = epsilon
), 'Range is: {} to {}. It must be -0.5 to 0.5'.format(
    np.min(normalized_features),
    np.max(normalized_features)
)

In [ ]:
print('Features shape:', normalized_features.shape)
print('Labels shape:', labels.shape)

# Design model

Use model by NVIDIA.

Source: https://arxiv.org/pdf/1604.07316v1.pdf

In [ ]:
width = normalized_features.shape[1]
length = normalized_features.shape[2]
depth = normalized_features.shape[3]

In [ ]:
# Add normalization layer
# Add dropout layer

In [ ]:
convolution_filter = 24
kernel_size = 5
stride_size = 2

model = Sequential()
model.add(Convolution2D(
    convolution_filter,
    kernel_size,
    kernel_size,
    border_mode = 'valid',
    subsample = (stride_size, stride_size),
    input_shape = (width, length, depth)
))

convolution_filter = 36

model.add(Convolution2D(
    convolution_filter,
    kernel_size,
    kernel_size,
    border_mode = 'valid',
    subsample = (stride_size, stride_size),
    input_shape = (width, length, depth)
))

convolution_filter = 48

model.add(Convolution2D(
    convolution_filter,
    kernel_size,
    kernel_size,
    border_mode = 'valid',
    subsample = (stride_size, stride_size),
    input_shape = (width, length, depth)
))

convolution_filter = 64
kernel_size = 3

model.add(Convolution2D(
    convolution_filter,
    kernel_size,
    kernel_size,
    border_mode = 'valid',
    input_shape = (width, length, depth)
))
model.add(Convolution2D(
    convolution_filter,
    kernel_size,
    kernel_size,
    border_mode = 'valid',
    input_shape = (width, length, depth)
))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

print('Shape:', model.output_shape)

# Train model

In [ ]:
# Training on a GPU is 20 times faster than CPU

In [ ]:
# Use transfer learning to refine a working model

In [ ]:
adam = Adam(lr = 0.000001)

In [ ]:
model.compile(optimizer = adam, loss = 'mse')
history = model.fit(normalized_features, labels, nb_epoch = EPOCH, validation_split = VALIDATION_SET_SIZE)

In [ ]:
# model.predict(normalized_features)

# Save model

In [ ]:
model_json = model.to_json()

json_file = open('model.json', 'w')
json_file.write(model_json)

In [ ]:
model.save_weights('model.h5')

In [ ]:
del model

# Load model

In [ ]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
model = model_from_json(loaded_model_json)

In [ ]:
model.load_weights('model.h5')

In [ ]:
model.summary()

# Evaluate model

Rubric: https://review.udacity.com/#!/rubrics/432/view

Experiment|Image|Set size|Optimizer|Learning rate|Epoch|Training time|Samples per second|Loss|Notes
-|
1|Center, normalized|256|Stochastic gradient descent|0.01|4|67 s|15.3|0.0194|Model may be overfitting as difference between training and validation loss increases per epoch. Loss becomes not a number when training model again.
2|Center, normalized|3|Stochastic gradient descent|0.000001|4|1 s|12|0.9166|Loss no longer becomes not a number due to reduced learning rate. Model predicts steering direction correctly.
3|Center, normalized|256|Stochastic gradient descent|0.000001|4|66 s|15.5|0.0533|Loss plateaus. Validation loss is greater than training loss. Car makes a hard left turn.
4|Center, normalized|512|Stochastic gradient descent|0.000001|4|134 s|15.3|0.0258|Loss plateaus. Training loss is greater than validation loss. Car makes a hard right turn.
5|Center, normalized|512|Adaptive movement estimation|0.000001|4|142 s|14.4|0.0266|Validation loss is greater than training loss. Car makes a hard right turn with brief hard left turns.

In [ ]:
print(history.history.keys())

In [ ]:
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.legend(['Training', 'Validation'])
pyplot.ylabel('Loss')
pyplot.xlabel('Epoch')
pyplot.show()